In [ ]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from google.colab import drive
import pathlib
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
directory  = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/petimages/dogscats')

In [ ]:
img_height = 150
img_width = 150
batch_size  = 32

In [ ]:
training_dataset = tf.keras.utils.image_dataset_from_directory(
  directory,
  validation_split=0.2,
  subset="training",
  seed=123,
  label_mode = 'int',
  shuffle = True,
  image_size=(img_height, img_width),
  batch_size=batch_size)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
  directory,
  validation_split=0.2,
  subset="validation",
  seed=123,
  shuffle = True,
  label_mode = 'int',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4000 files belonging to 2 classes.
Using 3200 files for training.
Found 4000 files belonging to 2 classes.
Using 800 files for validation.


In [ ]:
validation_batches = tf.data.experimental.cardinality(validation_dataset)
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
test_batches = validation_batches// 5
test_dataset = validation_dataset.take(test_batches)
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))
validation_dataset = validation_dataset.skip(test_batches)
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))

Number of validation batches: 25
Number of test batches: 5
Number of validation batches: 20


In [ ]:
class_names = training_dataset.class_names
class_names

['Cat', 'Dog']

In [ ]:
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.metrics import Precision, Recall

num_classes = len(class_names)

model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(img_width,img_height,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(num_classes, activation = 'softmax'))

In [ ]:
from keras import backend as K

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy', precision, recall])

In [ ]:
history = model.fit(x = training_dataset, validation_data = validation_dataset, epochs=10)

Epoch 1/10
 68/100 [===================>..........] - ETA: 1:25 - loss: 0.8642 - accuracy: 0.5823 - precision: 0.5060 - recall: 1.0000

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
precision =history.history['precision']
val_precision =history.history['val_precision']
recall =history.history['recall']
print('Loss:', loss[-1])
print('Accuracy: ', acc[-1])
print('Precision: ', precision[-1])
print('Recall: ', recall[-1])

Loss: 0.055502429604530334
Accuracy:  0.9803125262260437
Precision:  0.49687498807907104
Recall:  1.0


In [ ]:
model.evaluate(x = test_dataset)

5/5 [==============================] - 1s 69ms/step - loss: 0.9007 - accuracy: 0.7250 - precision: 0.5188 - recall: 1.0000


[0.9006646275520325, 0.7250000238418579, 0.518750011920929, 1.0]